In [ ]:
import atlite
from bc_power import utils,hydro
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os

### Cascaded basins inflows


In [ ]:
# # (0) create folder
# folder = r"/mnt/c/Users/pmcw9/Delta-E/PICS/PyPSA_BC/results/reservoir_inflows" #reservoir_inflows 
# utils.create_folder(folder)

In [ ]:
reservoir_sites.index.tolist()

In [ ]:
# (i) get configuration
config_file = r"/mnt/c/Users/pmcw9/Delta-E/PICS/PyPSA_BC/config/config.yaml"
cfg = utils.load_config(config_file)

# (i) read basin and site data
# Basins NA and artica
na_basin_data = hydro.load_hydro_basins(cfg["basin_files"]["na_file"])
ar_basin_data = hydro.load_hydro_basins(cfg["basin_files"]["artic_file"])
basin_data = gpd.GeoDataFrame(pd.concat([na_basin_data, ar_basin_data]))
cutout = atlite.Cutout(path=cfg["cutout"]["file"])

# Sites/plants which need inflow data.
reservoir_sites = hydro.load_reservoir_sites(cfg["reservoir_inflows"]["reservoir_assets"])

# # (ii) Calculated the inflows for each site
# # unmodified
# basins = hydro.prepare_basins(reservoir_sites, basin_data)

# basin_inflows = hydro.calculate_basin_inflows(basins, cutout,
#                                                height=bool(cfg['reservoir_inflows']['height']))

# basin_inflows_unmodified = hydro.calculate_basin_inflows(basins, cutout,
#                                                height=bool(cfg['reservoir_inflows']['height']))

# site_inflows_unmodified = hydro.calculate_plant_inflows(basin_inflows, basins,
#                                               flowspeed=cfg['reservoir_inflows']['flowspeed'])

# modified
basins_modified = hydro.prepare_cascade_basins(reservoir_sites, basin_data, cfg["reservoir_inflows"]["hydro_assets"])

basin_inflows_modified = hydro.calculate_basin_inflows(basins_modified, cutout,
                                               height=bool(cfg['reservoir_inflows']['height']))

site_inflows_modified = hydro.calculate_plant_inflows(basin_inflows_modified, basins_modified,
                                              flowspeed=cfg['reservoir_inflows']['flowspeed'])


In [ ]:
site_inflows = site_inflows_modified

### Normalize inflows by monthly measures

In [ ]:
def mean_inflow_normalize(site_inflows, fpath):
    '''
    This function is designed to take in a unnormalized inflow series and normalize it.
    The normalize is done for each month based on the mean monthly inflow in (cms) 
    fpath: Path to location of the inflow tables with statistics of historical inflows
           for BC reservoirs.
    '''
    # loop over reservoirs to normalize\
    # Final flow is in flow per hour
    
    month_2_num = {"January":1, "February":2, "March":3, "April":4, "May":5,
                    "June":6, "July":7, "August":8, "September":9,
                    "October":10, "November":11, "December":12}
    site_inflows_final = pd.DataFrame(index=pd.to_datetime([]))
    for rid in site_inflows.columns:
        read_path = fpath + f"/{rid}" + ".csv"
        if os.path.exists(read_path):
            df_inflow_stats = pd.read_csv(read_path)
        else:
            # Here is where imputation of zero can occur if 
            print(f'There are no flow statistics for {rid}')
            continue
        print('----------- START -------------')
        temp_series = pd.Series(dtype='float64')
        # loop over df_inflow_stats for this particular reservoir (month by month)
        for idx,row in df_inflow_stats.iterrows():
            month = row['Month'] # from row
            q_mean = row['Mean Monthly Inflow'] # from row

            # calculations
            mask = (site_inflows[rid].index.month == month_2_num[month]) 
            num_hours = site_inflows[rid].loc[mask].shape[0]
            const = site_inflows[rid].loc[mask].sum() / (3600*num_hours*q_mean)
            print(f"Reservoir: {rid} Month:{month} constant:{const}")
            # store to series,
            temp_series = pd.concat([temp_series, site_inflows[rid].loc[mask] / const])
        print('----------- END -------------')
        site_inflows_final[rid] = temp_series.sort_index()

    return site_inflows_final
    

def mean_inflow_constant(site_inflows, fpath):
    '''
    This function is designed to return a constant monthly inflow equal to
    that reservoirs historical mean inflow for each month.
    '''
    month_2_num = {"January":1, "February":2, "March":3, "April":4, "May":5,
                "June":6, "July":7, "August":8, "September":9,
                "October":10, "November":11, "December":12}
    site_inflows_final = pd.DataFrame(index=pd.to_datetime([]))

    for rid in site_inflows.columns:
        read_path = fpath + f"/{rid}" + ".csv"
        if os.path.exists(read_path):
            df_inflow_stats = pd.read_csv(read_path)
        else:
            print(f'There are no flow statistics for {rid}')
            continue
        temp_series = pd.Series(dtype='float64')
        # loop over df_inflow_stats for this particular reservoir (month by month)
        for idx,row in df_inflow_stats.iterrows():
            month = row['Month'] # from row
            q_mean = row['Mean Monthly Inflow'] # from row
            # calculations
            mask = (site_inflows[rid].index.month == month_2_num[month]) 
            temp_series = pd.concat([temp_series, site_inflows[rid].loc[mask].apply(lambda x: q_mean*3600)])

        site_inflows_final[rid] = temp_series.sort_index()

    return site_inflows_final


In [ ]:
# read in inflow tables
fpath = cfg['bc_hydro']['inflow_tables']
# final_inflows = mean_inflow_normalize(site_inflows, fpath)
final_inflows_const = mean_inflow_constant(site_inflows, fpath)


gen_sites = hydro.load_hydro_sites(cfg["reservoir_inflows"]["hydro_assets"])

# (1) Inflows for head upstream reservoirs in a cascade w/o WUP stats that are hydro_daily/run (nothing upstream of their upstream reservoir)
# These will be used
# preface: Already know no WUP stats + in a cascade + they are a head reservoir. Therefore, can target directly.
rid_list = ["BC_WHN_RES"]
inflow = pd.DataFrame(index=final_inflows_const.index) # This will be df of all RoR Inflows
for rid in rid_list:
    # Call RoR method and add this to it
    inflow_series[rid]

# (2) Inflows for downstream reservoirs in a cascade w/o WUP stats that are hydro_daily/run
# Will have an inflow time series of 0s 
# preface: Already know no WUP stats + in a cascade + they are not a head reservoir. Therefore, can target directly
rid_list = ["BC_PCN_RES","BC_STC_RES","BC_SFL_RES","BC_RUS_RES","BC_JHT_RES","BC_WAN_RES"]
for rid in rid_list:
    final_inflows_const[rid] = pd.Series([0]*final_inflows_const.shape[0],
                                dtype='float64', index=final_inflows_const.index)


In [ ]:
gen_sites['upper_reservoir_id']

In [ ]:
rid_list = reservoir_sites.index.tolist()
mask = gen_sites['upper_reservoir_id'].apply(lambda x: x in rid_list)


In [ ]:
gen_sites[mask]

In [ ]:
gen_sites.columns

In [ ]:
# check for WUp data
# rid = upper_reservoir_id
# (gen_sites['lower_reservoir_id'] == 'BC_MCA_RES').any()

In [ ]:
site_inflows.head(1)